## This is the Jupyter Notebook for the final project of my Data Science Immesive at General Assembly Austin Tx

#### Temporary Hypothesis, which features of my dataset best predict time on trail. 
## What sort of things can I add to this dataset? Weather,etc..
My data was obtained through the Appalachian Trail Conservancy. I called and asked for all availible data on Thru hikers. Talk about form. 

In [21]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [22]:
at = pd.read_excel('BenHikersInfo.xlsx')

In [23]:
def clean(df):
    df.drop(df.index[[17264,17236,17369]], inplace = True)
    df['Sex'] = df['Sex'].map({'M':0,'F':1})
    df['Finish'] = pd.to_datetime(df['Finish'])
    df['Start'] = pd.to_datetime(df['Start'])
    df.drop(labels = df[['Disease Other','School','DogWeightLbs','MinorityComments','School',
                         'DogBreed','Disability','Under2KAmount']], axis = 1, inplace = True)
    df.rename(columns = {'AgeYrFin': 'Age', 'Days Elapsed':'Days_On_Trail','Race or Heritage': 'Race',
                        'StAbbrev':'State', 'Hiking Disability ': 'Hiking_Disability', 
                         'Hike Type': 'Hike_Type', 'Yr Comp':'completion_yr', 'Hiking Disability':'Hiking_Disability'},inplace = True)
    df.dropna(subset= ['Start','Sex','Race','Country','Days_On_Trail','State','Hike_Type'],inplace = True)
    df['completion_yr'].fillna(2016,inplace = True)
    df['completion_yr'] = df['completion_yr'].astype(int)
    df['Days_On_Trail'] = df['Days_On_Trail'].astype(int)
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Total_Days'] = df['Finish'] - df['Start']
    df['Sex'] = df['Sex'].astype(int)
    
clean(at)

def clean_2(df):
    df.drop(labels = ['Days_On_Trail'], axis =1,inplace =True)
clean_2(at)

# drop hike types that are 'Alternative','alternative','Alterntive'
def clean_3(df):
    to_drop = ['Alternative','alternative','Alterntive']
    x = df[df['Hike_Type'].isin(to_drop)]
    y = x.index
    df.drop(y,inplace = True)
clean_3(at)




In [18]:
at.head()

,Finish,Age,Race,State,Sex,Country,Hike_Type,Start,completion_yr,Hiking_Disability,...,WinterHike,Under2KSpent,SchoolCredit,Minority,Vegetarian,DehydrateOwnFood,Giardia,Lyme,Norovirus,Total_Days
0,2016-09-28,27.000000,Caucasian or White,NH,1,USA,Section,2015-03-18,2016,NaN,...,False,False,False,False,False,False,False,False,False,560 days
8,1948-12-31,36.250254,Prefer not to say,PA,0,USA,NOBO,1948-01-01,1948,No,...,False,False,False,False,False,False,False,False,False,365 days
9,1951-10-10,28.000000,Prefer not to say,CT,0,USA,SOBO,1951-05-30,1951,No,...,False,False,False,False,False,False,False,False,False,133 days
10,1951-09-30,36.250254,Prefer not to say,GA,0,USA,NOBO,1951-05-31,1951,No,...,False,False,False,False,False,False,False,False,False,122 days
20,1959-09-17,20.000000,Caucasian or White,GA,0,USA,NOBO,1959-05-23,1959,No,...,False,False,False,False,False,False,False,False,False,117 days


In [19]:
at['Hike_Type'].value_counts()

NOBO       10522
Section     3099
SOBO        1503
Flip         914
Yo Yo          1
Name: Hike_Type, dtype: int64

In [25]:
at.shape


(16039, 21)

In [6]:
at.ix[396].to_frame().T


,Finish,Age,Race,State,Sex,Country,Hike_Type,Start,Days_On_Trail,completion_yr,...,WinterHike,Under2KSpent,SchoolCredit,Minority,Vegetarian,DehydrateOwnFood,Giardia,Lyme,Norovirus,Total_Days
396,1975-05-13 00:00:00,36.2526,Prefer not to say,NH,0,USA,Section,2015-01-01 00:00:00,-14477,1975,...,False,False,False,False,False,False,False,False,False,-14478 days +00:00:00


In [ ]:
at['Minority'].value_counts()

In [ ]:
at['Sex'].value_counts()

In [25]:
at[at['Sex'].str.contains('othe')]

AttributeError: Can only use .str accessor with string values, which use np.object_ dtype in pandas

In [ ]:
at[at['Sex'].str.contains('Oher')]

In [ ]:
df.drop([17264,17236])

In [ ]:
x['Sex'].str.contains('othe')

In [ ]:
at.shape

In [ ]:
at.isnull().sum().nlargest(30)

In [ ]:
#plt.figure(figsize = (10,10))
#null.plot(kind = 'bar')

# Notes on How I have cleaned my DataSet

1.) Days on trail was not accurate. So I dropped that column and created a new column subtracting finsih from start

2.) Convert the Start and Finish Dates to Pandas Datetime Objects

3.) Drop columns with null values that exceed 90 percent of the column 

4.) Drop Dog Weight Column 16238 of possible 17835 values were 0. So nearly the same as null 

5.) Drop Under2kAmount column 16125 of 17835 values were 0. It was just not self reported very well

4.) Rename Columns with spaces in them or w/ confusing names. 

5.) Replace Null Values in yr completed with 2016 since we do know his start date* 

6.) Drop oher and othe out of the sex column

7.) Convert null values in age column to mean. I checked out a distplot for how this would change
with using mean,median or mode, and not alot of change. 

8.) Dropped a column that had a finish date of 2022 as year 17369

9.)

10.) # Find out whether the section column included is a thru section. 

11.)

12.)

13.)

In [65]:
2.576 * .0087

0.0224112